## ① データのアップデート

In [17]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTy1ZbSq5OAVbEqScDw_Uf_KfLKydg0uUWBaBEPK_DXaBPCHmwv4ru9MzZHHnVHVmRVvLvOlndJ2ZNZ/pubhtml?gid=159569114&amp;single=true&amp;widget=true&amp;headers=false&amp;gid=0&amp;range=A:F%22'
res = requests.get(url)

In [3]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.text,'html.parser')

In [4]:
# 最新弾の情報を抽出する。
poke_new = soup.find('div',id ='1490875147')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_new.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,349):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(0)

#価格が欠損値になっている行を削除する。
df_new = df_new.dropna(subset = ['買取'])

#買取データの日付を取得する。
import datetime 
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_new.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [5]:
#### 通常の情報を抽出する。
poke_norm = soup.find('div',id ='159569114')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_norm.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,3644):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)

# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_norm.columns = df_norm.iloc[0]
df_norm = df_norm.drop(0)

#価格が欠損値になっている行を削除する。
df_norm = df_norm.dropna(subset = ['買取価格'])

#買取データの日付を取得する。
import datetime
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_norm.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [6]:
#更新日のデータを完成させる。
#データを結合する。
df_today = pd.concat([df_norm,df_new],ignore_index = True)
df_today.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [7]:
#集計データを読み込む。
df_preday = pd.read_csv('card_data.csv')

In [18]:
#結合のためのkeyを作る。(df_all)

df_today['key'] = df_today['カード名'].str.cat(df_today['型番'],sep = ' , ',na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['仕様'],sep = ' ,' ,na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['R'],sep = ' ,' ,na_rep='')

# print('df_todayのユニークなkeyの個数:',len(df_today['key'].unique()))
# print('df_todayのデータの列数:',len(df_today))
# print('df_todayのkeyに重複のあるデータ:\n',df_today[df_today['key'].duplicated()])

In [19]:
#結合のためのkeyを作る。(df_all)

df_preday['key'] = df_preday['カード名'].str.cat(df_preday['型番'],sep = ' , ',na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['仕様'],sep = ' ,' ,na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['R'],sep = ' ,' ,na_rep='')

# print('df_predayのユニークなkeyの個数:',len(df_preday['key'].unique()))
# print('df_predayのデータの列数:',len(df_preday))
# print('df_predayのkeyに重複のあるデータ:\n',df_preday[df_preday['key'].duplicated()])

In [10]:
#keyが被る行にtodayの数値を代入する。

df_preday[today] = 0
df_preday

for i in range(len(df_preday)):
    for j in range(len(df_today)):
        if df_preday.loc[i,'key'] == df_today.loc[j,'key']:
            df_preday.loc[i,today] = df_today.loc[j,today]
            break

In [11]:
count = 0
for i in range(len(df_today)):
    flg = 0
    for j in range(len(df_preday)):
        if df_today.loc[i,'key'] == df_preday.loc[j,'key']:
            flg = 1
            break
            
    if flg == 0:
        df_preday = pd.concat([df_preday,df_today[i:i+1]])
        df_preday.reset_index(drop = True,inplace=True)
        count = count + 1

In [20]:
#データの整理を行う。
df_update = df_preday
df_update = df_update.drop('key',axis = 1)

#データの上書きを行う。
df_update.to_csv('card_data.csv',index = None)

## ② データの分析

In [21]:
#数値をint型に変換する。
df_update = df_update.replace(',','', regex=True)

#欠損値の処理を行う。
df_update.fillna({'カード名':'-','仕様':'-','型番':'-','タイプ':'-'},inplace = True)
df_update.fillna(0,inplace = True)
df_update = pd.concat([df_update.iloc[:,:4],df_update.iloc[:,5:].astype(int)],axis = 1)

#前日との値差を計算する。
from datetime import date, timedelta
today = datetime.date.today()
yesterday = today - timedelta(days = 1) 

today = today.strftime('%y-%m-%d')
yesterday = yesterday.strftime('%y-%m-%d')

df_update['前日との値差'] = df_update[today] - df_update[yesterday]  

In [42]:
#値上がりしたカードを抽出する。
print('■ 前日からの値上がりしたカード')
up_num = len(df_update[df_update['前日との値差'] > 0])
pd.set_option('display.max_rows',up_num)
df_update.sort_values('前日との値差',ascending=False)[0:up_num]

■ 前日からの値上がりしたカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,前日との値差
3466,ロイヤルマスク,SR仕様,P,085/SM-P,2300000,2300000,2500000,200000
3419,シロナの想い,-,P,-,350000,350000,400000,50000
3467,アセロラ,SR仕様,P,395/SM-P,250000,250000,280000,30000
3469,リーリエ,SR仕様,P,397/SM-P,480000,480000,500000,20000
1853,ピカチュウ,20th アニバーサリーフェスタ,P,XY-P,110000,110000,130000,20000
101,リザードンGX,-,SSR,209/150,40000,40000,45000,5000
3468,グズマ,SR仕様,P,396/SM-P,45000,45000,50000,5000
1828,ピカチュウ,-,UR,056/053,15000,12000,15000,3000
1829,ピカチュウ,-,UR,056/053,12000,12000,15000,3000
436,ピカチュウVMAX,RRR仕様,P,123/S-P,14000,14000,16000,2000


In [47]:
print('■ 前日からの値下がりが大きいカード')
down_num = len(df_update[df_update['前日との値差'] < 0])
pd.set_option('display.max_rows',down_num)
df_update.sort_values('前日との値差',ascending=True).head(down_num)

■ 前日からの値下がりが大きいカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,前日との値差
3545,ミュウツーEX,-,P,XY-P,65000,65000,0,-65000
3552,R団のミュウツーex,-,R,064/084,5000,5000,0,-5000
3547,ミュウツーEX,-,SR,055/052,4000,4000,0,-4000
3551,ミュウツーex,-,R,026/055,3500,3500,0,-3500
623,あなぬけのヒモ,-,UR,062/051,17000,17000,15000,-2000
3548,ミュウツーEX,-,P,190/XY-P,1000,1000,0,-1000
3549,ミュウツーEX,-,P,191/XY-P,1000,1000,0,-1000
3624,コルニの気合い,-,SR,079/070,6500,6500,5800,-700
3573,いちげきウーラオスVMAX,-,HR,084/070,3000,2800,2300,-500
3618,フヨウ,-,SR,080/070,3800,3800,3300,-500


In [48]:
#新規に値段がついたカードを表示する。
print('■ 新規にデータを追記したカード')
df_update.tail(count)

■ 新規にデータを追記したカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,前日との値差
3664,ビリリダマ,-,-,003/033,0,0,10,10
3665,マルマイン,-,-,004/033,0,0,100,100
3666,ドーミラー,-,C,053/070,0,0,10,10
3667,ネズ,-,U,095/100,0,0,10,10
3668,ネズ,-,-,018/033,0,0,10,10
3669,オーロラエネルギー,-,U,060/060,0,0,10,10
3670,オーロラエネルギー,-,-,187/190,0,0,10,10
3671,ヨマワル,-,C,023/064,0,0,10,10
3672,電磁レーダー,-,U,043/055,0,0,10,10
3673,電磁レーダー,-,-,123/173,0,0,10,10
